In [1]:
import torch
from torch import Tensor
from torch.autograd import Variable
from torch import nn

from helpers import *

%load_ext autoreload
%autoreload 2

In [2]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci')
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


In [2]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci', one_khz = True)
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False, one_khz = True)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [3]:
train_input[0]


  41.8000   44.8000   47.1000  ...    69.8000   72.6000   76.1000
 -10.3000   -5.9000   -3.3000  ...    12.6000   24.0000   26.5000
  38.1000   25.2000   46.0000  ...    45.1000   74.1000   64.8000
             ...                ⋱                ...             
   7.9000   11.2000   14.3000  ...    32.7000   43.4000   45.5000
  19.2000   33.6000   33.8000  ...    46.7000   53.7000   43.4000
  -0.4000   12.7000   12.0000  ...    30.7000   40.6000   33.1000
[torch.FloatTensor of size 28x50]

In [4]:
train_target


 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 316]

## Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
logistic_reg = LogisticRegression()

In [7]:
logistic_reg.fit(train_input.view(train_input.size(0),-1),train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
pred = logistic_reg.predict(test_input.view(test_input.size(0),-1))

In [9]:
100*(1-(torch.LongTensor(pred) - test_target).abs().sum()/test_input.size(0))

74.0

## Neural Network

In [38]:
net = nn.Sequential(
        nn.Linear(1400, 100),
        nn.ReLU(),
        nn.Linear(100, 500),
        nn.ReLU(),
        nn.Linear(500, 1000),
        nn.ReLU(),
        nn.Linear(1000, 2)
)

In [40]:
train_model(net, Variable(train_input.view(train_input.size(0),-1)), Variable(train_target))

Variable containing:
 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
[torch.LongTensor of size 100]

Variable containing:
 5.6689 -1.9097
 1.3716 -0.9696
 3.0073 -0.5094
 3.9367 -0.7384
 3.7865 -1.0175
 5.9726 -1.6857
 1.7181 -0.6752
 5.7647 -1.6978
 2.7314  0.1496
 3.7525  0.0489
 5.6494 -1.2135
 2.0555 -0.3021
 4.1995 -0.5168
 4.8598  0.0988
 4.8479 -1.2264
 1.9205  0.0481
 4.1898 -0.7582
 7.8047 -1.0712
 1.7113 -0.7956
 7.2884 -2.8951
 5.6126 -1.3367
 6.6588 -1.7152
 2.2302 -0.6097
 1.5378 -0.3204
 2.1236 -1.3282
 5.3400 -0.9369
 1.7680 -0.8475
 1.5322 -0.3163
 3.3102 -0.8842
 2.0830 -1.0325
 6.0704 -0.6842
 4.3024 -0.7684
 4.2964 -1.1051
 5.6996 -0.9989
 2.3173 -1.0217
 4.5447 -0.8977
 3.9744  0.5899
 3.6724  0.5041
 5.0730 -1.4584
 3

RuntimeError: multi-target not supported at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/THNN/generic/ClassNLLCriterion.c:22